# 1.1 Amazon SageMaker 기본 Training 실습

## 학습 작업의 실행 노트북 개요
이 노트북은 주요하게 아래의 작업을 수행 합니다.

- 로컬 머신에서 로컬 모드 및 클라우드 모드로 모델 훈련
    - 환결 설정은 아래 링크를 참조 하세요.
        - [로컬 머신에서 Visual Studio Code 를 사용하여 SageMaker 훈련 코드 작성 하기](https://github.com/gonsoomoon-ml/Self-Study-On-SageMaker/blob/main/Environment/README-Local-VS-Code.md)
- SageMaker Notebook 에서 로컬 모드 및 클라우드 모드로 모델 훈련


이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)




### 작업 실행 시 필요 라이브러리 import

In [1]:
import boto3
import sagemaker

### Bucket 정의

In [2]:
use_default_bucket = True

if use_default_bucket:
    bucket = sagemaker.session.Session().default_bucket()
else:
    bucket = '<Type your bucket name here>'
    
print("bucket: ", bucket)    

bucket:  sagemaker-us-east-1-057716757052


###  훈련 잡에 사용할 SageMaker Role 정의 
- 로컬 머신에서 사용할 경우에는 아래의 주석을 해제하시고, SageMaker Execution Role ARN 을 넣어 주세요.

In [3]:
use_local_machine = False
# use_local_machine = True

if use_local_machine:
    # role = '<Type Role ARN here>'
    pass    
else:
    role = sagemaker.get_execution_role()    


### 로컬 모드 혹은 클라우드 모드에 따른 SageMaker 세션 정의

In [4]:
use_local_mode = False
# use_local_mode = True

if use_local_mode:
    instance_type = "local"
    # instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.m5.large"
    



### 하이퍼파라미터 정의

In [5]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 데이터셋 정의

In [6]:
if use_local_mode:
    local_data_path = "../data/dataset/"
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('1_lab_1_xgboost')[0] + 'data/dataset/'
else:
    data_path=f's3://{bucket}/xgboost/dataset'
    !aws s3 sync ../data/dataset/ $data_path    
    inputs = data_path

print("input data path: ", inputs)                

input data path:  s3://sagemaker-us-east-1-057716757052/xgboost/dataset


### 학습 실행 작업 정의

In [7]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir='src',
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count= 1,
    instance_type=instance_type,
    framework_version="1.3-1",
)

### 학습 실행

In [8]:
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

In [9]:
%%time 

estimator.logs()

2023-05-08 09:55:07 Starting - Starting the training job...
2023-05-08 09:55:30 Starting - Preparing the instances for trainingProfilerReport-1683539707: InProgress
......
2023-05-08 09:56:30 Downloading - Downloading input data...
2023-05-08 09:57:02 Training - Downloading the training image.....[2023-05-08 09:57:48.008 ip-10-0-207-102.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-05-08 09:57:48.032 ip-10-0-207-102.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-05-08:09:57:48:INFO] Imported framework sagemaker_xgboost_container.training
[2023-05-08:09:57:48:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-08:09:57:48:INFO] Invoking user training script.
[2023-05-08:09:57:48:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel 

### 데이터 세트 설명
- 데이터 세트는 블로그 [Architect and build the full machine learning lifecycle with AWS: An end-to-end Amazon SageMaker demo](https://aws.amazon.com/blogs/machine-learning/architect-and-build-the-full-machine-learning-lifecycle-with-amazon-sagemaker/) 에서 사용한 데이터를 사용합니다. 블로그에서는 데이터 세트에 대해서 이렇게 설명 합니다.
- "자동차 보험 청구 사기를 탐지를 위해서 블로그 저자가 데이터를 합성해서 만든 고객과 클레임의 데이터 세트를 사용합니다."

In [10]:
import pandas as pd

In [11]:
train_prep_df = pd.read_csv('../data/dataset/train.csv')
train_prep_df.groupby('fraud').sample(n=5)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
2750,0,41110.469673,63310.469673,47,97,0,1,750,3000,98027,...,0,0,0,1,0,0,0,1,0,1
3915,0,17996.316404,27696.316404,41,139,0,1,750,3000,83714,...,1,1,0,0,0,0,0,1,1,0
1191,0,17858.742518,27058.742518,52,123,0,1,750,3000,85308,...,0,1,0,0,1,0,0,0,1,0
1393,0,13928.504573,26028.504573,42,50,0,2,750,2800,99344,...,0,1,0,0,1,0,0,0,1,0
839,0,22294.303419,25194.303419,24,60,0,1,750,3000,93442,...,0,1,0,0,0,0,0,1,0,1
3409,1,4199.525396,9999.525396,34,52,0,2,750,2800,95926,...,0,0,1,0,0,0,1,0,1,0
2567,1,14975.838073,53775.838073,53,19,0,4,750,2750,98023,...,1,0,1,0,0,0,0,1,1,0
3614,1,40163.331842,49163.331842,48,48,0,2,750,3000,90006,...,0,0,0,1,0,0,0,1,0,1
1522,1,11661.704259,19361.704259,22,32,0,3,750,3000,94583,...,1,1,0,0,0,0,0,1,1,0
414,1,3079.236859,4179.236859,32,55,0,1,750,3000,94703,...,1,1,0,0,0,0,0,1,1,0


In [12]:
train_prep_df.groupby('fraud').size()

fraud
0    3869
1     131
dtype: int64

#### 데이터 컬럼 설명
- fraud: 보험 청구의 사기 여부 입니다. 1 이면 사기, 0 이면 정상 청구 입니다.
- vehicle_claim: 자동차에 대한 보험 청구액. 값으로서, $1000, $17,638 등이 있습니다.
- total_claim_amount: 전체 보험 청구액 입니다. $21,400, $10,000 등이 있습니다.    
- customer_age: 고객의 나이를 의미 합니다.
- months_as_customer: 고객으로서의 가입 기간을 의미합니다. 단위는 월로서 11, 30, 31 등의 값이 존재 합니다.
- num_claims_past_year: 작년의 보험 청구 수를 의미 합니다. 0, 1, 2, 3, 4, 5, 6 의 값이 존재 합니다.
- num_insurers_past_5_years: 과거 5년 동안의 보험 가입 회사 수를 의미 합니다. 1, 2, 3, 4, 5 의 값이 존재 합니다.
- policy_deductable: 보험의 최소 자기 부담금 입니다. $750, $800 등이 있습니다.    
- policy_annual_premium: 보험의 특약 가입에 대한 금액 입니다. $2000, $3000 등이 있습니다.
- customer_zip: 고객의 집 주소 우편 번호를 의미합니다.
- auto_year: 자동차의 년식을 의미 합니다. 2020, 2019 등이 있습니다.
- num_vehicles_involved: 몇 대의 자동차가 사고에 연관 되었는지 입니다. 1, 2, 3, 4, 5, 6 의 값이 있습니다.
- num_injuries: 몇 명이 상해를 입었는지를 기술합니다. 0, 1, 2, 3, 4, 의 값이 있습니다.
- num_witnesses: 몇 명의 목격자가 있었는지를 기술합니다. 0, 1, 2, 3, 4, 5 의 값이 있습니다.
- injury_claim: 상해에 대한 보험 청구액. \$5,500, \$70,700, \$100,700 등이 있습니다.    
- incident_month: 사고가 발생한 월을 의미합니다. 1~12 값이 존재 합니다.
- incident_day: 사고가 발생한 일자를 의미합니다. 1~31 값이 존재 합니다.
- incident_dow: 사고가 발생한 요일을 의미합니다. 0~6 값이 존재 합니다.
- incident_hour: 사고가 발생한 시간을 의미합니다. 0~23 값이 존재 합니다.
- policy_state: 보험 계약을 한 미국 주(State)를 의미 합니다. CA, WA, AZ, OR, NV, ID 가 존재 합니다.    
- policy_liability: 보험 청구의 한도를 의미 합니다. 예를 들어서 25/50 은  사람 당 상해 한도 $25,000, 사고 당 상해 한도가 $50,000 을  의미합니다. 25/50, 15/30, 30/60, 100/200 의 값이 존재 합니다. 
- customer_gender: 고객의 성별을 의미 합니다. Male, Female, Unkown, Other가 존재 합니다.
- customer_education: 고객의 최종 학력을 의미합니다. Bachelor, High School, Advanced Degree, Associate, Below High School 이 존재 합니다.
- driver_relationship: 보험 계약자와 운전자와의 관계 입니다. Self, Spouse, Child, Other 값이 존재 합니다.
- incident_type: 사고의 종류를 기술합니다. Collision, Break-in, Theft 값이 존재 합니다.
- collision_type: 충돌 타입을 기술합니다. Front, Rear, Side, missing 값이 존재 합니다.
- incident_severity: 사고의 손실 정도 입니다. Minor, Major, Totaled 값이 존재 합니다.
- authorities_contacted: 어떤 관련 기관에 연락을 했는지 입니다. Police, Ambuylance, Fire, None 값이 존재 합니다.
- police_report_available: 경찰 보고서가 존재하는지를 기술합니다. Yes, No 의 값이 있습니다.